# **1. ติดตั้งโปรแกรมที่เกี่ยวข้อง**

In [1]:
#@title **1.1.Install packages and dependencies_updated**
#@markdown Thanks to **`mamba`**, the installation takes **less than 5 mins**. \

#@markdown Please note the software version:
#@markdown + Python 3.7.12
#@markdown + Mamba 1.4.2
#@markdown + py3Dmol 2.0.0
#@markdown + pybel 0.15.5
#@markdown + rdkit-pypi 2022.9.5
#@markdown + pdb2pqr 3.5.2
#@markdown + biconda
#@markdown + mgltools 1.5.7
#@markdown + biopython 1.78
#@markdown + openbabel 3.1.0
#@markdown + plip 2.2.2
#@markdown + zlib 1.2.11
#@markdown + xlsxwriter 3.0.3
#@markdown + Autodock vina 1.2.3
#@markdown + smina
#@markdown + Quick vina 2.1





# %%capture
import contextlib
import timeit
import torch

# -- Start install --
start = timeit.default_timer()

with open("/content/installation.log", "w") as i:
    with contextlib.redirect_stdout(i):
        !wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.12.0-Linux-x86_64.sh
        !chmod +x Miniconda3-py37_4.12.0-Linux-x86_64.sh
        !bash ./Miniconda3-py37_4.12.0-Linux-x86_64.sh -b -f -p /usr/local
        import sys
        sys.path.append('/usr/local/lib/python3.7/site-packages/')
        !conda install -c conda-forge mamba -y

        #pip
        !pip install py3Dmol==2.0.0.post2
        !pip install pybel==0.15.5
        !pip install rdkit-pypi==2022.9.5
        !pip install pdb2pqr==3.5.2
        import os
        os.chdir('/content')
        !git clone https://github.com/tieulongphan8995/dock_util.git


        # Vina
        !wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.3/vina_1.2.3_linux_x86_64 -O vina
        !chmod u+x vina

        # qvina2
        %cd /content
        !git clone https://github.com/QVina/qvina.git
        %cd /content/qvina/bin
        !chmod u+x qvina2.1
        %cd /content

        #smina
        !wget https://downloads.sourceforge.net/project/smina/smina.static
        !chmod u+x smina.static
        !./smina.static --help



        # install conda
        os.chdir('/content')
        !mamba install -c conda-forge -c bioconda mgltools=1.5.7 biopython=1.78 \
          openbabel=3.1.0 plip=2.2.2 zlib=1.2.11 xlsxwriter=3.0.3 -y

        !rm -r /content/sample_data /content/condacolab_install.log

stop = timeit.default_timer()
# -- End install --

time_elp = stop - start

print(f"+ Operation completed")
print(f"+ Time elapsed: {int(time_elp//60):02}m {int(time_elp%60):02}s")

+ Operation completed
+ Time elapsed: 04m 11s


In [2]:
#@title **1.2. Import Python modules**

import os
import sys
sys.path.append('/content/dock_util')
from grid_box import *
from result_process import *


import ast
import math
import plip
import timeit
import shutil
import py3Dmol
import contextlib
import xlsxwriter
import urllib.request

import numpy as np
import pandas as pd

from google.colab import drive, files
from tqdm.notebook import tqdm
from openbabel import pybel
from Bio.PDB import PDBIO, PDBParser
from rdkit import Chem
from rdkit.Chem import rdFMCS, AllChem, PandasTools
from plip.exchange.report import BindingSiteReport
from plip.structure.preparation import PDBComplex
#from AutoDockTools.Utilities24 import prepare_ligand4, prepare_receptor4
print(f"+ Imported done")
print(f"+ Environment ready for molecular docking")

No module Pymol
+ Imported done
+ Environment ready for molecular docking


In [26]:
#@title **1.3. Create folders**
#@markdown Enter a **< Job Name >** without space.\
#@markdown This create a folder for protein, ligand, experimental and docking.

Job_name = "7RFW" #@param {type:"string"}
assert not Job_name == "", "Do not leave this blank."
assert not any(c == "/" or c == "." for c in Job_name), "Disallowed characters."

DIR = os.getcwd()
WRK_DIR = os.path.join(DIR, Job_name)
PRT_FLD = os.path.join(WRK_DIR, "PROTEIN")
LIG_FLD = os.path.join(WRK_DIR, "LIGAND")
EXP_FLD = os.path.join(WRK_DIR, "EXPERIMENTAL")
DCK_FLD = os.path.join(WRK_DIR, "DOCKING")
INT_FLD = os.path.join(WRK_DIR, "INTERACTION")

folders = [WRK_DIR, PRT_FLD, LIG_FLD, EXP_FLD, DCK_FLD, INT_FLD]

for f in folders:
    if os.path.exists(f):
        print(f"+ {os.path.basename(f)} folder already exists")
    if not os.path.exists(f):
        os.mkdir(f)
        print(f"+ {os.path.basename(f)} folder created")

+ 7RFW folder created
+ PROTEIN folder created
+ LIGAND folder created
+ EXPERIMENTAL folder created
+ DOCKING folder created
+ INTERACTION folder created


In [27]:
#@title **1.4. Set up utilities**
#@markdown This creates important variables and functions that will be utilized
#@markdown throughout the docking study.
import os

%alias vina /content/vina
%alias qvina2 /content/qvina/bin/qvina2.1
%alias smina ./smina.static


COLORS = ["red", "orange", "yellow", "lime", "green", "cyan", "teal", "blue",
          "violet", "purple", "pink", "gray", "brown", "white", "black"]

BOND_DICT = {"hydrophobic": ["0x59e382", "GREEN"],
             "hbond": ["0x59bee3", "LIGHT BLUE"],
             "waterbridge": ["0x4c4cff", "BLUE"],
             "saltbridge": ["0xefd033", "YELLOW"],
             "pistacking": ["0xb559e3", "PURPLE"],
             "pication": ["0xe359d8", "VIOLET"],
             "halogen": ["0x59bee3", "LIGHT BLUE"],
             "metal": ["0xe35959", "ORANGE"]}




import os
import os
import sys


import ast
import math
import plip
import timeit
import shutil
import py3Dmol
import contextlib
import xlsxwriter
import urllib.request

import numpy as np
import pandas as pd

from google.colab import drive, files
from tqdm.notebook import tqdm
from openbabel import pybel
from Bio.PDB import PDBIO, PDBParser
from rdkit import Chem
from rdkit.Chem import rdFMCS, AllChem, PandasTools
from plip.exchange.report import BindingSiteReport
from plip.structure.preparation import PDBComplex

COLORS = ["red", "orange", "yellow", "lime", "green", "cyan", "teal", "blue",
          "violet", "purple", "pink", "gray", "brown", "white", "black"]

BOND_DICT = {"hydrophobic": ["0x59e382", "GREEN"],
             "hbond": ["0x59bee3", "LIGHT BLUE"],
             "waterbridge": ["0x4c4cff", "BLUE"],
             "saltbridge": ["0xefd033", "YELLOW"],
             "pistacking": ["0xb559e3", "PURPLE"],
             "pication": ["0xe359d8", "VIOLET"],
             "halogen": ["0x59bee3", "LIGHT BLUE"],
             "metal": ["0xe35959", "ORANGE"]}

class Hide:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# Biological assembly detector and extractor
def chain_profilier(inputFile):
    parser = PDBParser()
    structure = parser.get_structure("X", inputFile)
    chain_obj = [s for s in structure.get_chains()]
    chain_IDs = [c.get_id() for c in chain_obj]
    chain_len = len(chain_obj)
    return [chain_len, chain_IDs, chain_obj]

def seperate_chain(inputFile):
    io = PDBIO()
    name = os.path.basename(inputFile)[:-4]
    dir = os.path.dirname(inputFile)
    chain_info = chain_profilier(inputFile)
    chain_len, chain_IDs, chain_obj = chain_info[0], chain_info[1], chain_info[2]
    print(f"+ Chains detected: {chain_len} ({', '.join(chain_IDs)})")
    for ID,chain in zip(chain_IDs, chain_obj):
        chain_pdb = name + "_" + ID + ".pdb"
        chain_pdb_file = os.path.join(dir, chain_pdb)
        io.set_structure(chain)
        io.save(chain_pdb_file)
        print(f"+ {chain_pdb} ==> {os.path.basename(dir)} folder")

# Binding interactions profiler
def interaction_profiler(inputPL):
    protlig = PDBComplex()
    INTER = {}
    INTER_TYPE = []
    ID = os.path.basename(inputPL[:-7])
    CSV_Ifile = os.path.join(INT_FLD, f"{ID[:-2]}/{ID}_inter.csv")
    COL = ["RESNR", "RESTYPE", "RESCHAIN", "RESNR_LIG", "RESTYPE_LIG",
           "RESCHAIN_LIG", "DIST", "LIGCARBONIDX", "PROTCARBONIDX", "LIGCOO",
           "PROTCOO"]
    INTER_DF = pd.DataFrame(columns=COL)

    protlig.load_pdb(inputPL)
    LIG = protlig.ligands[0]
    protlig.characterize_complex(LIG)

    BD_keys = BOND_DICT.keys()
    BSI_key = f":".join([str(x) for x in LIG.members[0]])
    BSI_val = protlig.interaction_sets[BSI_key]
    BSR = BindingSiteReport(BSI_val)
    INTER[BSI_key] = {k: [getattr(BSR, f"{k}_features")] +
                      getattr(BSR, f"{k}_info")
                      for k in tuple(BD_keys)}
    for BD_key in list(BD_keys):
        DF = pd.DataFrame.from_records(
            INTER[BSI_key][BD_key][1:],
            columns=INTER[BSI_key][BD_key][0])
        if not DF.empty:
            INTER_TYPE.extend([BD_key.upper()]*len(DF))
            INTER_DF = INTER_DF.append(DF)
    INTER_DF = INTER_DF.assign(BOND=INTER_TYPE)
    INTER_DF.reset_index(drop=True, inplace=True)
    INTER_DF.to_csv(CSV_Ifile, index=False)

def view_interaction(inputCSV, result="summary"):
    DIST_CALC = []
    COLOR = []
    MIDCOO = []
    interaction = pd.read_csv(inputCSV, converters={
        "LIGCOO": lambda x: ast.literal_eval(str(x)),
        "PROTCOO": lambda x: ast.literal_eval(str(x)),
        "BOND": lambda x: x.lower()})
    for LC, PC, BT in zip(interaction["LIGCOO"], interaction["PROTCOO"],
                          interaction["BOND"]):
        COLOR.append(BOND_DICT[BT][1])
        p1 = np.array([LC[0], LC[1], LC[2]])
        p2 = np.array([PC[0], PC[1], PC[2]])
        squared_dist = np.sum((p1-p2)**2, axis=0)
        dist = np.round(np.sqrt(squared_dist) ,2)
        DIST_CALC.append(dist)
        mid_x = np.round((LC[0] + PC[0]) / 2, 2)
        mid_y = np.round((LC[1] + PC[1]) / 2, 2)
        mid_z = np.round((LC[2] + PC[2]) / 2, 2)
        p_mid = (mid_x, mid_y, mid_z)
        MIDCOO.append(p_mid)
    interaction["BOND"] = interaction["BOND"].str.upper()
    interaction["COLOR"] = COLOR
    interaction["MIDCOO"] = MIDCOO
    interaction["DIST_CALC"] = DIST_CALC
    interaction["RESNAME"] = interaction["RESTYPE"] + interaction["RESNR"].astype(str)

    summary = ["RESNR", "RESTYPE", "DIST_CALC", "BOND", "COLOR"]
    py3dmol = ["RESNR", "DIST_CALC", "LIGCOO", "PROTCOO", "MIDCOO", "BOND"]
    if result == "summary":
        df = interaction[summary]
    if result == "py3Dmol":
        df = interaction[py3dmol]
    return df

# Py3Dmol model viewer
def VIEW_PROT(inputP, model="Cartoon", color="white", focusRes="",
              focusResColor="white", addStick=False, addLine=False,
              showChain=False, showResLabel=False, showVDW=False,
              outline=False):
    # Variables
    mview = py3Dmol.view(1000,1500)
    if model in "none":
        model = cType = color = ""
    if model in "cartoon":
        cType = "color"
    if model in ("line", "stick"):
        cType, color = "colorscheme", color + "Carbon"

    # Protein Model
    count = 1
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    mol = open(inputP, "r").read()
    mview.addModel(mol, "pdb")
    mview.setStyle(
        {"model": prot_model},
        {model: {cType: color}})

    # Show chains
    chainLen = chain_profilier(inputP)[0]
    if showChain and chainLen > 1:
        for n,c in zip(range(chainLen), COLORS):
            mview.setStyle(
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]},
                {model: {cType: c if model == "cartoon" else c + "Carbon"}})
            mview.addLabel(
                f"Chain {chr(65+n)}",
                {"fontColor": c, "backgroundOpacity": 0.7, "alignment": "topLeft"},
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]})

    # Focus RES
    if focusRes != "":
        res = focusRes.split(",")
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"stick": {"colorscheme": focusResColor + "Carbon", "radius": 0.15}})
        # Label focused RES
        if showResLabel:
            mview.addResLabels(
                {"and": [{"model": prot_model}, {"resi": res}]},
                {"alignment": "bottomLeft",
                 "showBackground": False,
                 "inFront": True,
                 "fixed": False,
                 "fontSize": 14,
                 "fontColor": "0x000000",
                 "screenOffset": {"x": 15, "y": 15}})

    # Show VDW surface
    if showVDW:
        mview.addSurface(
            py3Dmol.VDW,
            {"color": "white", "opacity": 0.4},
            {"model": prot_model})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": prot_model})
    mview.show()

def VIEW_ELIG(inputE, showAtomLabel=False, outline=False):
    # Variable
    mview = py3Dmol.view(1000, 1500)

    # Experimental ligand model
    count = 1
    elig_model = count
    print(f"+ Showing {os.path.basename(inputE)}")
    mol = open(inputE, "r").read()
    mview.addModel(mol, "pdb")
    mview.setStyle(
        {"model": elig_model},
        {"stick": {"colorscheme": "lightGrayCarbon"}})

    # Label all atoms
    if showAtomLabel:
        mview.addPropertyLabels(
            "atom",
            {"model": elig_model},
            {"backgroundOpacity": 0.7, "inFront": True})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.zoomTo({"model": elig_model})
    mview.show()

def VIEW_LIG(inputL, showHs=False, showAtomLabel=False, outline=False):
    # Variable
    mview = py3Dmol.view(1000, 1500)

    # Ligand model
    count = 1
    lig_model = count
    print(f"+ Showing {os.path.basename(inputL)}")
    smi = open(inputL, "r").read()
    mol = Chem.MolFromSmiles(smi)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)

    # Remove Hs
    if not showHs:
        mol = Chem.RemoveHs(mol)
    mblock = Chem.MolToMolBlock(mol)
    mview.addModel(mblock, "mol")
    mview.setStyle(
        {"model": lig_model},
        {"stick": {"colorscheme": "lightGrayCarbon"}})

    # Label all atoms
    if showAtomLabel:
        mview.addPropertyLabels(
            "atom",
            {"model": lig_model},
            {"backgroundOpacity": 0.7, "inFront": True})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": lig_model})
    mview.show()

def VIEW_GRID(inputP, inputE, focusRes, center, size=[10, 10, 10]):
    # Variable
    mview = py3Dmol.view(1000, 1500)

    # Grid box
    bxi, byi, bzi = center[0], center[1], center[2]
    bxf, byf, bzf = size[0], size[1], size[2]
    print(f"+ Center: X {center[0]}  Y {center[1]}  Z {center[2]}")
    print(f"+ Size: W {size[0]}  H {size[1]}  D {size[2]}")
    mview.addBox(
        {"center":{"x":bxi, "y":byi, "z":bzi},
        "dimensions": {"w":bxf, "h":byf, "d":bzf},
        "color": "skyBlue", "opacity": 0.7})

    # Protein model
    count = 1
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    molA = open(inputP, "r").read()
    mview.addModel(molA, "pdb")
    mview.setStyle(
        {"model": prot_model},
        {"cartoon": {"color": "white"}})

    # Experimental ligand model
    count += 1
    elig_model = count
    print(f"+ Showing {os.path.basename(inputE)}")
    molB = open(inputE, "r").read()
    mview.addModel(molB, "pdb")
    mview.setStyle(
        {"model": elig_model},
        {"stick": {"colorscheme": "greenCarbon"}})

    # Focus RES
    if focusRes != "":
        res = focusRes.split(",")
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"stick": {"colorscheme": "orangeCarbon", "radius": 0.15}})
        mview.addResLabels(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"alignment": "bottomLeft",
            "showBackground": False,
            "inFront": True,
            "fixed": False,
            "fontSize": 14,
            "fontColor": "0x000000",
            "screenOffset": {"x": 15, "y": 15}})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": elig_model})
    mview.show()

def VIEW_PILE(inputP, inputPP, inputL, inputE, inputCSV, model="Cartoon",
              color="white", focusRes="", focusResColor="white",
              showInter=False, showSubunit=False, showResLabel=False,
              showVDW=False, showPartProt=False, showExpLig=False,
              showLig=False, showAllPose=False, showBestPose=False,
              outline=False):
    # Variables
    resUQ = []
    mview = py3Dmol.view(1000,1500)
    if model in "none":
        model = cType = color = ""
    if model in "cartoon":
        cType = "color"
    if model in ("line", "stick"):
        cType, color = "colorscheme", color + "Carbon"

    # Protein model
    count = 1
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    molA = open(inputP, "r").read()
    mview.addModel(molA, "pdb")
    mview.setStyle(
        {"model":prot_model},
        {model: {cType: color}})

  # Show chains
    chainLen = chain_profilier(inputP)[0]
    if showSubunit and chainLen > 1:
        for n, c in zip(range(chainLen), COLORS):
            mview.setStyle(
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]},
                {model: {cType: c if model == "cartoon" else c + "Carboon"}})
            mview.addLabel(
                f"Subunit {chr(65+n)}",
                {"fontColor": c, "backgroundOpacity": 0.7, "alignment": "topLeft"},
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]})

    # Focus RES
    if focusRes != "":
        resUQ.extend([int(r) for r in focusRes.split(",")])
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": resUQ}]},
            {"stick": {"colorscheme": focusResColor + "Carbon", "radius": 0.15}})
        # Label focused RES
        if showResLabel:
            mview.addResLabels(
                {"and": [{"model": prot_model}, {"resi": resUQ}]},
                {"alignment": "bottomLeft",
                 "showBackground": False,
                 "inFront": True,
                 "fixed": False,
                 "fontSize": 14,
                 "fontColor": "0x000000",
                 "screenOffset": {"x": 15, "y": 15}})

    # Interactions
    if showInter:
        interaction = view_interaction(inputCSV, result="py3Dmol")
        RESNR = interaction["RESNR"]
        DIST_CALC = interaction["DIST_CALC"]
        LIGCOO = interaction["LIGCOO"]
        PROTCOO = interaction["PROTCOO"]
        MIDCOO = interaction["MIDCOO"]
        BOND = interaction["BOND"]
        for RN,DC,LC,PC,MC,BT in zip(RESNR, DIST_CALC, LIGCOO, PROTCOO, MIDCOO, BOND):
            BT = BT.lower()
            if RN not in resUQ:
                resUQ.append(RN)
                mview.addStyle(
                    {"and": [{"model": prot_model}, {"resi": RN}]},
                    {"stick": {"colorscheme": "whiteCarbon", "radius": 0.15}})
                mview.addResLabels(
                    {"and": [{"model": prot_model}, {"resi": RN}]},
                    {"alignment": "bottomLeft", "showBackground": False,
                     "inFront": True, "fixed": False, "fontSize": 14,
                     "fontColor": "0x000000", "screenOffset": {"x": 15, "y": 15}})
                mview.addCylinder(
                    {"start": {"x": LC[0], "y": LC[1], "z": LC[2]},
                     "end": {"x": PC[0], "y": PC[1], "z": PC[2]},
                     "radius": 0.05, "fromCap": 1, "toCap": 1,
                     "color": BOND_DICT[BT][0], "dashed": True})
                mview.addLabel(
                    str(DC) + " Å",
                    {"position": {"x": MC[0], "y": MC[1], "z": MC[2]},
                     "alignment": "bottomLeft", "inFront": False, "fixed": False,
                     "backgroundColor": BOND_DICT[BT][0], "fontSize": 10,
                     "screenOffset": {"x": 5, "y": 5}})

    # Show VDW surface
    if showVDW:
        mview.addSurface(
            py3Dmol.VDW,
            {"color": "white", "opacity": 0.4},
            {"model": prot_model})

    # Partner protein model
    if showPartProt and os.path.exists(inputPP):
        count += 1
        pprot_model = count
        print(f"+ Showing {os.path.basename(inputPP)} (yellow)")
        molB = open(inputPP, "r").read()
        mview.addModel(molB, "pdb")
        mview.setStyle(
            {"model": pprot_model},
            {"cartoon": {"color": "yellow"}})
        mview.addStyle(
            {"model": pprot_model},
            {"stick": {"colorscheme": "yellowCarbon"}})

    # Experimental ligand model
    if showExpLig:
        count += 1
        elig_model = count
        print(f"+ Showing {os.path.basename(inputE)} (gray)")
        molC = open(inputE, "r").read()
        mview.addModel(molC, "pdb")
        mview.setStyle(
            {"model": elig_model},
            {"stick": {"color": "gray"}})

    # Ligand model
    if showLig:
        count += 1
        lig_model = count
        print(f"+ Showing {os.path.basename(inputL)} (orange)")
        molD = open(inputL, "r").read()
        mview.addModel(molD, "pdb")
        mview.setStyle(
            {"model": lig_model},
            {"stick": {"colorscheme": "orangeCarbon"}})

    # Show poses of selected ligand
    if showAllPose:
        pose = sorted([ os.path.join(os.path.dirname(inputL), f) for f in os.listdir(os.path.dirname(inputL)) if f.endswith(".pdb") ])
        pose.remove(inputL[len(DCK_FLD)+1:]) if inputL[len(DCK_FLD)+1:] in pose else None
        for f in pose:
            count += 1
            pose_model = count
            fs = "".join(f)
            mol5 = open(os.path.join(DCK_FLD, fs), "r").read()
            mview.addModel(mol5, "pdb")
            mview.setStyle(
                {"model": pose_model},
                {"stick": {"color": "blue", "opacity": 0.5, "radius": 0.2}})

    if outline:
        mview.setViewStyle(
            {"style": "outline", "color": "black", "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": lig_model})
    mview.enableFog(True)
    mview.show()


# **2.ดาวน์โหลดโครงสร้างสามมิติโปรตีนเป้าหมาย (Target protein structure preparation) จากฐานข้อมูล RCSB Protein Data Bank (PDB)**

In [28]:
#@title **2.1. Generate protein PDB file**
#@markdown Enter the **< PDB ID >** to download targeted protein.

PDB_ID = "7RFW" # @param {type:"string"}
PDB_pdb = PDB_ID + ".pdb"
PDB_pdb_Pfile = os.path.join(PRT_FLD, PDB_pdb)

urllib.request.urlretrieve(f"http://files.rcsb.org/download/{PDB_pdb}", PDB_pdb_Pfile)
print(f"+ PDB downloaded: {PDB_pdb} ==> PROTEIN folder")

+ PDB downloaded: 7RFW.pdb ==> PROTEIN folder


In [29]:
#@title **2.2. Extract protein**
#@markdown This generate **clean** protein structure for molecular docking.\
#@markdown The protein chains will be extracted into separate files.

PDB_prot = PDB_ID + "_prot"
PDB_prot_pdb = PDB_prot + ".pdb"
PDB_prot_pdb_Pfile = os.path.join(PRT_FLD, PDB_prot_pdb)

selected = ["ATOM"]
with open(PDB_prot_pdb_Pfile, "w") as o, open(PDB_pdb_Pfile, "r") as i:
    for line in i:
        row = line.split()
        if any(a == row[0] for a in selected):
            o.write(line)

print(f"+ Protein extracted: {PDB_pdb} --> {PDB_prot_pdb}")
seperate_chain(PDB_prot_pdb_Pfile)

+ Protein extracted: 7RFW.pdb --> 7RFW_prot.pdb
+ Chains detected: 1 (A)
+ 7RFW_prot_A.pdb ==> PROTEIN folder


In [30]:
#@title **2.3. Display 3D protein** { run: "auto" }
#@markdown Enter the **< Protein >** to be viewed.

View = "7RFW_prot_A" #@param {type:"string"}
Model = "cartoon" #@param ["none", "line", "stick",  "cartoon"]
Model_colour = "yellow" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Focus_residues = "" #@param {type:"string"}
Residues_colour = "red" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Show_all_chains = True #@param {type:"boolean"}
Show_residues_label = True #@param {type:"boolean"}
Show_VDW_surface = False #@param {type:"boolean"}
View_outline = False #@param {type:"boolean"}
pview_Pfile = os.path.join(PRT_FLD, f"{View}.pdb")

VIEW_PROT(inputP=pview_Pfile,
          model=Model,
          color=Model_colour,
          focusRes=Focus_residues,
          focusResColor=Residues_colour,
          showChain=Show_all_chains,
          showResLabel=Show_residues_label,
          showVDW=Show_VDW_surface,
          outline=View_outline)

+ Showing 7RFW_prot_A.pdb



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [31]:
#@title **2.5. Parameterise protein with Gasteiger charges**
#@markdown Enter the **< Protein >** to be parameterised.\
#@markdown This generate a **`protein.pdbqt`** file after:
#@markdown + Addition of Gasteiger Partial Charge
#@markdown + pH 7.4
#@markdown + Removal of non-polar hydrogens

Target_protein = "7RFW_prot_A" #@param {type:"string"}
PROT_pdb = Target_protein + ".pdb"
PROT_pqr = Target_protein + ".pqr"
PROT_pqr_Pfile = os.path.join(PRT_FLD, PROT_pdb)
PROT_pqr_Dfile = os.path.join(DCK_FLD, PROT_pqr)

PROT_pdbqt = Target_protein + ".pdbqt"
PROT_pdb_Pfile = os.path.join(PRT_FLD, PROT_pdb)
PROT_pdb_Dfile = os.path.join(DCK_FLD, PROT_pdb)
PROT_pdbqt_Dfile = os.path.join(DCK_FLD, PROT_pdbqt)

with Hide():
  while(True):
    try:
      !pdb2pqr30 --ff AMBER --keep-chain --titration-state-method propka --with-ph 7.4 $PROT_pqr_Pfile $PROT_pqr_Dfile
      !python2 "/usr/local/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py" -r $PROT_pqr_Dfile -o $PROT_pdbqt_Dfile -A hydrogens -U nphs_lps -v
      break
    except:
      print('Can not handle missing residues')
      !python2 "/usr/local/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_receptor4.py" -r $PROT_pqr_Pfile -o $PROT_pdbqt_Dfile -A hydrogens -U nphs_lps -v

  #if os.path.isdir(PROT_pqr_Dfile) == False:

  shutil.copy(PROT_pdb_Pfile, PROT_pdb_Dfile)

print(f"+ Protein parameterised: {PROT_pdb} --> {PROT_pdbqt}")
print(f"+ {PROT_pdb} ==> DOCKING folder")
print(f"+ {PROT_pdbqt} ==> DOCKING folder")

+ Protein parameterised: 7RFW_prot_A.pdb --> 7RFW_prot_A.pdbqt
+ 7RFW_prot_A.pdb ==> DOCKING folder
+ 7RFW_prot_A.pdbqt ==> DOCKING folder


In [32]:
#@title **2.6. Generate experimental ligand PDB file**: Extracting the experimental ligand for later comparison with the binding modes of docked ligand.
#@markdown Enter the **< Keyword >** assigned to query for the experimental
#@markdown ligand in the **`pdb`** fle.

Keyword = "4WI" #@param {type:"string"}
Elig_pdb = Keyword + ".pdb"
Elig_pdb_Efile = os.path.join(EXP_FLD, Elig_pdb)

selected = [Keyword]
with open(Elig_pdb_Efile, "w") as o, open(PDB_pdb_Pfile, "r") as i:
    for line in i:
        row = line.split()
        if any(a in row for a in selected):
            o.write(line)

print(f"+ Experimental ligand extracted: {PDB_pdb} --> {Elig_pdb}")

+ Experimental ligand extracted: 7RFW.pdb --> 4WI.pdb


In [33]:
#@title **2.7. Extract corystal ligands**
#@markdown This generate **clean** corystal ligands for comparison.\
#@markdown The experimental ligands will be extracted into separate files.

seperate_chain(Elig_pdb_Efile)

+ Chains detected: 1 (A)
+ 4WI_A.pdb ==> EXPERIMENTAL folder


In [37]:
#@title **2.8. Display 3D ligand** {run: "auto"}
#@markdown Enter the **< Corystal Ligand >** to be viewed.

View = "4WI_A" #@param {type:"string"}
Show_atom_labels = True #@param {type:"boolean"}
View_outline = True #@param {type:"boolean"}
eview_Efile = os.path.join(EXP_FLD, f"{View}.pdb")

VIEW_ELIG(inputE=eview_Efile,
          showAtomLabel=Show_atom_labels,
          outline=View_outline)

+ Showing 4WI_A.pdb



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [38]:
#@title **2.9. Choose corystal ligand**
#@markdown Enter the **< Target corystal Ligand >** for later comparison.

Target_experimental_ligand = "4WI_A" #@param {type:"string"}
EXP_pdb = Target_experimental_ligand + ".pdb"
EXP_pdb_Efile = os.path.join(EXP_FLD, EXP_pdb)
EXP_pdb_Dfile = os.path.join(DCK_FLD, EXP_pdb)

with Hide():
    shutil.copy(EXP_pdb_Efile, EXP_pdb_Dfile)

print(f"+ {EXP_pdb} ==> DOCKING folder")

+ 4WI_A.pdb ==> DOCKING folder


# **3. ดาวน์โหลดโครงสร้าง ลิแกนด์ที่สนใจจากฐานข้อมูล (Preparing the Ligand)**

In [39]:
#@title **3.1. Generate SMILES file**
#@markdown Enter the ligand **< ID >** and **< SMILES >** to be docked. \
#@markdown This generate **`smi`** file for the ligand.

Ligand_ID = "1RP" #@param {type:"string"}
Ligand_SMILES = "C1=C(N=C(C(=O)N1[C@H]2[C@@H]([C@@H]([C@H](O2)COP(=O)(O)O)O)O)C(=O)N)F" #@param {type:"string"}

LIG_ID = Ligand_ID
LIG_smi = LIG_ID + ".smi"
LIG_mol2 = LIG_ID + ".mol2"
LIG_pdbqt = LIG_ID + ".pdbqt"
LIG_smi_Lfile = os.path.join(LIG_FLD, f"{LIG_smi}")
LIG_mol2_Lfile = os.path.join(LIG_FLD, f"{LIG_mol2}")
LIG_pdbqt_Dfile = os.path.join(DCK_FLD, f"{LIG_ID}/{LIG_pdbqt}")

with open(LIG_smi_Lfile, "w") as o:
    o.write(Ligand_SMILES)

print(f"+ {LIG_smi} ==> LIGAND folder")

+ 1RP.smi ==> LIGAND folder


In [41]:
#@title **3.2. Display 3D ligand** {run: "auto"}
#@markdown Enter the **< Ligand >** to be viewed.

View = "1RP" #@param {type:"string"}
Show_atom_labels = False #@param {type:"boolean"}
Show_hydrogens = True #@param {type:"boolean"}
View_outline = False #@param {type:"boolean"}
lview_Lfile = os.path.join(LIG_FLD, f"{View}.smi")

VIEW_LIG(inputL=lview_Lfile,
         showHs=Show_hydrogens,
         showAtomLabel=Show_atom_labels,
         outline=View_outline)

+ Showing 1RP.smi



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [42]:
#@title **3.3. Parameterize ligand(s) with Gasteiger charges**
#@markdown Select **< Force Field >** for energy minimization. \
#@markdown This generate **`ligand.mol2`** and **`ligand.pdbqt`** file after:
#@markdown + Energy minization at 10,000 iterations
#@markdown + Addition of Gasteiger Partial Charge
#@markdown + Removal of non-polar hydrogens

# NOTE: The prepare_ligand4.py does not recognise the mol2 file location in
#       LIGAND folder for some reason. Hence, the mol2 file was generated in
#       /content/ and then move back into LIGAND folder after parameterization.

Select_force_field = "GAFF" #@param ["GAFF", "Ghemical", "MMFF94", "MMFF94s", "UFF"]

LIG_Dfld = os.path.join(DCK_FLD, f"{Ligand_ID}")
LIG_mol2_DRfile = os.path.join(DIR, f"{LIG_mol2}")
os.makedirs(LIG_Dfld, exist_ok=True)

with Hide():
    !obabel $LIG_smi_Lfile -O $LIG_mol2_DRfile --gen3d --OB_RANDOM_SEED 42 --best --canonical \
    --minimize --ff $Select_force_field --steps 10000 --sd
    !python2 "/usr/local/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_ligand4.py" -l $LIG_mol2_DRfile -o $LIG_pdbqt_Dfile -U nphs_lps -v
    shutil.move(LIG_mol2_DRfile, LIG_mol2_Lfile)

print(f"+ {LIG_smi} ==> LIGAND folder")
print(f"+ {LIG_pdbqt} ==> DOCKING folder")

+ 1RP.smi ==> LIGAND folder
+ 1RP.pdbqt ==> DOCKING folder


---
---
# **4. กำหนดค่าพารามิเตอร์สำหรับการทำ Molecular Docking ZSetting Up Molecular Docking)**

We then define search space on the target protein through the use of grid box, which often centered within the binding, active or allosteric site of target protein. The size should be sufficiently enough such that important binding residues are contained inside the box. Methods to define grid box include:

+ **`LABOGRID`** = Utilize the min and max XYZ coordinates of ligand [🔗](https://github.com/RyanZR/labogrid)
+ **`eBoxSize`** = Utilize the radius of gyration of ligand [🔗](https://github.com/michal-brylinski/eboxsize)
+ **`eBoxSize-modifed`** = Utilize `eBoxSize` but box center is calculated with `LABOGRID` method
+ **`Autodock-Grid`** = 22.5 × 22.5 × 22.5 Å
+ **`manual`** = Use the slider below to define grid box


In [43]:
#@title **4.1. Place gridbox at binding site** { run: "auto" }
#@markdown Place the **< Gridbox >** at the center of binding site. \

Focus_residues = "" #@param {type:"string"}
Method = "Autodock-Grid" #@param ["LABOGRID", "eBoxSize", "eBoxSize-modified", "Autodock-Grid", "manual"]

#@markdown ---

if Method == "LABOGRID":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = labogrid(atomCoord)
    Center, Size = grid[0], grid[1]

if Method == "eBoxSize":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = eboxsize(atomCoord)
    Center, Size = grid[0], grid[1]

if Method == "eBoxSize-modified":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = eboxsize_mod(atomCoord)
    Center, Size = grid[0], grid[1]

if Method == "Autodock-Grid":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = autodock_grid(atomCoord)
    Center, Size = grid[0], grid[1]

#@markdown **Manual Mode**
if Method == "manual":
    X = 143.399 #@param {type:"slider", min:-200, max:200, step:1}
    Y = 159.402 #@param {type:"slider", min:-200, max:200, step:1}
    Z = 132 #@param {type:"slider", min:-200, max:200, step:1}
    Width = 22.5 #@param {type:"slider", min:0, max:30, step:1}
    Height = 22.5 #@param {type:"slider", min:0, max:30, step:1}
    Depth = 22.5 #@param {type:"slider", min:0, max:30, step:1}
    Scale_factor = 1 #@param {type:"slider", min:0.1, max:5.0, step:0.1}
    Center = [X, Y, Z]
    Size = [Width*Scale_factor, Height*Scale_factor, Depth*Scale_factor]

VIEW_GRID(inputP=PROT_pdb_Dfile,
          inputE=EXP_pdb_Dfile,
          focusRes=Focus_residues,
          center=Center,
          size=Size)

+ Center: X 9.9  Y 1.715  Z 20.552
+ Size: W 22.5  H 22.5  D 22.5
+ Showing 7RFW_prot_A.pdb
+ Showing 4WI_A.pdb



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [44]:
#@title **4.2. Generate docking config file**
#@markdown This generate **Config File** for AutoDock Vina.

CONF = "config_file"
CONF_Dfile = os.path.join(DCK_FLD, CONF)

with open(CONF_Dfile, "w") as o:
    o.write(f"center_x = {Center[0]} \n")
    o.write(f"center_y = {Center[1]} \n")
    o.write(f"center_z = {Center[2]} \n")
    o.write(f"\n")
    o.write(f"size_x = {Size[0]} \n")
    o.write(f"size_y = {Size[1]} \n")
    o.write(f"size_z = {Size[2]} \n")

print(f"+ {CONF} ==> DOCKING folder" )

+ config_file ==> DOCKING folder


---
---
# **5. ขั้นตอนการจำลองการจับกันเชิงโมเลกุล (Molecular Docking)**

Finally, we use Autodock Vina to perform the molecular docking on the target ligand. The process may take longer than 2 minutes, depending on the number of rotatable bond and level of exhaustiveness.



In [45]:
#@title **5.1. Run Docking**
#@markdown Select **< Exhaustiveness >** for molecular docking. (Default: 8)\
#@markdown Autodock Vina will performs molecular docking on targeted protein.\
#@markdown This generate **`ligand_output.pdbqt`**.

#@markdown **NOTE:** Compound with many rotatable bonds may required higher
#@markdown level of exhaustiveness to ensure accurate and consistent pose
#@markdown prediction.
Select_Docking_tools = "Autodock Vina" #@param ["Autodock Vina",  "Qvina2", "smina"]


Exhaustiveness = 8 #@param {type:"slider", min:8, max:200, step:1}
ex = Exhaustiveness

Number_of_conformations = 10 #@param {type:"slider", min:10, max:1000, step:1}
mode = Number_of_conformations

number_of_cpu = 0 #@param {type:"slider", min:-1, max:32, step:1}
n_cpu = number_of_cpu




LIG_out_pdbqt = LIG_ID + "_output.pdbqt"
LIG_out_sdf = LIG_ID + "_output.sdf"
LIG_log_txt = LIG_ID + "_log.txt"
LIG_out_pdbqt_Dfile = os.path.join(DCK_FLD, f"{LIG_ID}/{LIG_out_pdbqt}")
LIG_out_sdf_Dfile = os.path.join(DCK_FLD, f"{LIG_ID}/{LIG_out_sdf}")
LIG_log_txt_Dfile = os.path.join(DCK_FLD, f"{LIG_ID}/{LIG_log_txt}")







# -- Start docking --
start = timeit.default_timer()
with open(LIG_log_txt_Dfile, "w") as i:
    with contextlib.redirect_stdout(i):
      if Select_Docking_tools == 'Autodock Vina':
        %vina --receptor $PROT_pdbqt_Dfile --ligand $LIG_pdbqt_Dfile \
        --out $LIG_out_pdbqt_Dfile --config $CONF_Dfile \
        --exhaustiveness $ex --num_modes $mode --seed 42 --cpu $n_cpu
      elif Select_Docking_tools == 'Qvina2':
        %qvina2 --receptor $PROT_pdbqt_Dfile --ligand $LIG_pdbqt_Dfile \
        --out $LIG_out_pdbqt_Dfile --config $CONF_Dfile \
        --exhaustiveness $ex --num_modes $mode --seed 42 --cpu $n_cpu
      elif Select_Docking_tools == 'smina':
        %smina --receptor $PROT_pdbqt_Dfile --ligand $LIG_pdbqt_Dfile \
        --out $LIG_out_pdbqt_Dfile --config $CONF_Dfile \
        --exhaustiveness $ex --num_modes $mode --seed 42 --cpu $n_cpu

with open(LIG_log_txt_Dfile, "r") as r:
    data = r.read().splitlines(True)
with open(LIG_log_txt_Dfile, "w") as o:
    o.writelines(data[1:])
stop = timeit.default_timer()
# -- End docking --

time_elp = stop - start

print(f"+ Operation completed")
print(f"+ Time elapsed: {int(time_elp//60):02}m {int(time_elp%60):02}s")


+ Operation completed
+ Time elapsed: 01m 27s


In [46]:
#@title **5.2. Process output file**
#@markdown This generate **`ligand_output.sdf`** for comparison with experimental ligand.\
#@markdown This also generate **`ligand.pdb`** for each binding modes.

input = pybel.readfile("pdbqt", LIG_out_pdbqt_Dfile)
output = pybel.Outputfile("sdf", LIG_out_sdf_Dfile, True)
LIG_out_d_pdb_Dfile = os.path.join(DCK_FLD, f"{LIG_ID}/{LIG_ID}_.pdb")

mols = [m for m in input]
for mol in mols:
    mol.data.update({"POSE": mol.data["MODEL"]})
    mol.data.update({"SCORE": mol.data["REMARK"].split()[2]})
    mol.data.update({"RMSD_LB_BP": mol.data["REMARK"].split()[3]})
    mol.data.update({"RMSD_UB_BP": mol.data["REMARK"].split()[4]})
    del mol.data["MODEL"], mol.data["REMARK"], mol.data["TORSDO"]
    output.write(mol)
output.close()

with Hide():
    !obabel $LIG_out_pdbqt_Dfile -O $LIG_out_d_pdb_Dfile -m

print(f"+ {LIG_out_sdf} ==> DOCKING folder")
print(f"+ {len(mols)} {LIG_ID}.pdb ==> DOCKING folder")

+ 1RP_output.sdf ==> DOCKING folder
+ 10 1RP.pdb ==> DOCKING folder


---
---
# **6. วิเคราะห์ผลการการเข้าจับของลิแกนด์กับโปรตีนเป้าหมาย (Protein-Ligand Interaction Identifier)**

We will use PLIP to determine the binding interaction between the docked ligands and target proteins.

In [47]:
#@title **6.1. Generate protein-ligand complex file**
#@markdown This merge ligand pose and protein into a **`PL.pdb`** file. \
#@markdown Required by **PLIP** to profile binding interactions.

LIG_Dfile = os.listdir(LIG_Dfld)
LIG_Ifld = os.path.join(INT_FLD, f"{LIG_ID}")
os.makedirs(LIG_Ifld, exist_ok=True)

N = 0
for n in tqdm(range(len(LIG_Dfile)-4)):
    pose_pdb_Dfile = os.path.join(DCK_FLD, f"{LIG_ID}/{LIG_ID}_{n+1}.pdb")
    complex_pdb_Ifile = os.path.join(INT_FLD, f"{LIG_ID}/{LIG_ID}_{n+1}_PL.pdb")
    prot_data = open(PROT_pdb_Dfile, "r").readlines()
    pose_data = open(pose_pdb_Dfile, "r").readlines()

    with open(complex_pdb_Ifile, "w") as o:
        for line in prot_data[:-1]:
            o.write(line)
        for line in pose_data:
            word = line.split()
            if word[0] == "ATOM" or word[0] == "CONECT":
                o.write(line)
        o.write("END")
    N += 1

print(f"+ {N} {LIG_ID}_PL.pdb ==> INTERACTION folder")

  0%|          | 0/10 [00:00<?, ?it/s]

+ 10 1RP_PL.pdb ==> INTERACTION folder


In [48]:
#@title **6.2. Profile protein-ligand interactions**
#@markdown This identify **binding interactions** between docked ligand and target protein. \
#@markdown All interactions data is exported as **`inter.csv`** file.
def interaction_profiler(inputPL):
    protlig = PDBComplex()
    INTER = {}
    INTER_TYPE = []
    ID = os.path.basename(inputPL[:-7])
    CSV_Ifile = os.path.join(INT_FLD, f"{ID[:-2]}/{ID}_inter.csv")
    COL = ["RESNR", "RESTYPE", "RESCHAIN", "RESNR_LIG", "RESTYPE_LIG",
           "RESCHAIN_LIG", "DIST", "LIGCARBONIDX", "PROTCARBONIDX", "LIGCOO",
           "PROTCOO"]
    INTER_DF = pd.DataFrame(columns=COL)

    protlig.load_pdb(inputPL)
    LIG = protlig.ligands[0]
    protlig.characterize_complex(LIG)

    BD_keys = BOND_DICT.keys()
    BSI_key = f":".join([str(x) for x in LIG.members[0]])
    BSI_val = protlig.interaction_sets[BSI_key]
    BSR = BindingSiteReport(BSI_val)
    INTER[BSI_key] = {k: [getattr(BSR, f"{k}_features")] +
                      getattr(BSR, f"{k}_info")
                      for k in tuple(BD_keys)}
    for BD_key in list(BD_keys):
        DF = pd.DataFrame.from_records(
            INTER[BSI_key][BD_key][1:],
            columns=INTER[BSI_key][BD_key][0])
        if not DF.empty:
            INTER_TYPE.extend([BD_key.upper()]*len(DF))
            INTER_DF = INTER_DF.append(DF)
    INTER_DF = INTER_DF.assign(BOND=INTER_TYPE)
    INTER_DF.reset_index(drop=True, inplace=True)
    INTER_DF.to_csv(CSV_Ifile, index=False)

LIG_Dfile = os.listdir(LIG_Dfld)

N = 0
for n in tqdm(range(len(LIG_Dfile)-4)):

    complex_pdb_Ifile = os.path.join(INT_FLD, f"{LIG_ID}/{LIG_ID}_{n+1}_PL.pdb")
    print(complex_pdb_Ifile)
    interaction_profiler(complex_pdb_Ifile)
    N += 1


print(f"+ {N} {LIG_ID}_inter.CSV ==> INTERACTION folder")

  0%|          | 0/10 [00:00<?, ?it/s]

/content/7RFW/INTERACTION/1RP/1RP_1_PL.pdb


<ipython-input-48-234fe47113fb>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INTER_DF = INTER_DF.append(DF)
<ipython-input-48-234fe47113fb>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INTER_DF = INTER_DF.append(DF)
<ipython-input-48-234fe47113fb>:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  INTER_DF = INTER_DF.append(DF)
<ipython-input-48-234fe47113fb>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INTER_DF = INTER_DF.append(DF)


/content/7RFW/INTERACTION/1RP/1RP_2_PL.pdb


<ipython-input-48-234fe47113fb>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INTER_DF = INTER_DF.append(DF)
<ipython-input-48-234fe47113fb>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INTER_DF = INTER_DF.append(DF)
<ipython-input-48-234fe47113fb>:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  INTER_DF = INTER_DF.append(DF)


...

/content/7RFW/INTERACTION/1RP/1RP_9_PL.pdb


<ipython-input-48-234fe47113fb>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INTER_DF = INTER_DF.append(DF)
<ipython-input-48-234fe47113fb>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INTER_DF = INTER_DF.append(DF)
<ipython-input-48-234fe47113fb>:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  INTER_DF = INTER_DF.append(DF)


/content/7RFW/INTERACTION/1RP/1RP_10_PL.pdb


<ipython-input-48-234fe47113fb>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INTER_DF = INTER_DF.append(DF)
<ipython-input-48-234fe47113fb>:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  INTER_DF = INTER_DF.append(DF)
<ipython-input-48-234fe47113fb>:32: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  INTER_DF = INTER_DF.append(DF)


OSError: ignored

---
---
# **7. วิเคราะห์ค่าพลังงานหลังจากการทำ Molecular Docking (Analysis of Docking Results)**

The final step is to evaluate the docking score and visualize the 3D binding mode. Note that docking score is essentially predicted binding affinity.

In [49]:
#@title **7.1. Generate docking results**
#@markdown This generate **scoring result** of docked ligands in **`csv`** format.

EXP_pose = Chem.MolFromPDBFile(EXP_pdb_Dfile)
EXP_pose.SetProp("_Name", os.path.basename(EXP_pdb_Dfile)[:-4])

LIG_result_csv_Dfile = os.path.join(DCK_FLD, f"{LIG_ID}/{LIG_ID}_result.csv")
data = pd.DataFrame(get_score(LIG_out_sdf_Dfile, EXP_pose, result="full"))
data.to_csv(LIG_result_csv_Dfile, index=False)

print(f"+ {LIG_ID}_result.csv ==> DOCKING folder")

+ 1RP_result.csv ==> DOCKING folder


In [50]:
#@title **7.2. Show docking result of ligand** {run : "auto"}
#@markdown Noted that:
#@markdown + **`RMSD_EL`** = RMSD vs Exp. Lig.
#@markdown + **`RMSD_LB_BP`** = RMSD Lower Bound vs Best Pose
#@markdown + **`RMSD_LB_UP`** = RMSD Upper Bound vs Best Pose

ligResultDF = pd.read_csv(LIG_result_csv_Dfile)
ligResultDF

,ID,POSE,SCORE,RMSD_EL,RMSD_LB_BP,RMSD_UB_BP
0,1RP_1,1,-7.355,4.492,0.000,0.000
1,1RP_2,2,-7.169,6.029,3.076,4.246
2,1RP_3,3,-7.007,4.685,2.148,2.691
3,1RP_4,4,-6.785,4.021,2.990,3.995
4,1RP_5,5,-6.714,4.639,2.321,3.515
5,1RP_6,6,-6.645,3.555,4.000,6.568
6,1RP_7,7,-6.583,4.875,3.711,6.633
7,1RP_8,8,-6.583,3.563,4.278,6.800
8,1RP_9,9,-6.539,4.491,3.223,4.939
9,1RP_10,10,-6.518,5.647,2.485,3.698


In [54]:
#@title **7.3. Analyze conformations** {run : "auto"}

View_ligand = "1RP" #@param {type:"string"}
View_experimental_ligand = "4WI_A" #@param {type:"string"}
View_protein = "7RFW_prot_A" #@param {type:"string"}


pview_Dfile = os.path.join(DCK_FLD, f"{View_protein}.pdb")
lview_Dfile = os.path.join(DCK_FLD, f"{View_ligand[:]}/{View_ligand}_output.sdf")
eview_Dfile = os.path.join(DCK_FLD, f"{View_experimental_ligand}.pdb")

v = py3Dmol.view()
v.addModel(open(pview_Dfile).read())
v.setStyle({'cartoon':{},'stick':{'radius':.1}})
v.addModel(open(eview_Dfile).read())
v.setStyle({'model':1},{'stick':{'colorscheme':'redCarbon','radius':.125}})
v.addModelsAsFrames(open(lview_Dfile).read())
v.setStyle({'model':2},{'stick':{'colorscheme':'greenCarbon'}})
v.animate({'interval':1000})
v.zoomTo({'model':1})
v.rotate(90)


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [73]:
#@title **7.4. Show interaction profile of ligand** {run: "auto"}

Select_ligand = "1RP_1" #@param {type : "string"}
LIG_inter_CSV_Ifile = os.path.join(INT_FLD, f"{Select_ligand[:-2]}/{Select_ligand}_inter.csv")
view_interaction(LIG_inter_CSV_Ifile, result="summary")

,RESNR,RESTYPE,DIST_CALC,BOND,COLOR
0,54,TYR,4.00,HBOND,LIGHT BLUE
1,142,ASN,2.97,HBOND,LIGHT BLUE
2,166,GLU,2.88,HBOND,LIGHT BLUE
3,54,TYR,3.26,HBOND,LIGHT BLUE
4,163,HIS,4.65,SALTBRIDGE,YELLOW
5,41,HIS,5.48,PISTACKING,PURPLE


In [80]:
#@title **7.5. Display 3D binding mode** { run: "auto" }

#@markdown **PROTEIN MODEL**
View_protein = "7RFW_prot_A" #@param {type:"string"}
Model = "cartoon" #@param ["none", "line", "stick",  "cartoon"]
Model_colour = "white" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Focus_residues = "200" #@param {type:"string"}
Residues_colour = "white" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Show_all_subunits = False #@param {type:"boolean"}
Show_residues_label = True #@param {type:"boolean"}
Show_interaction = True #@param {type:"boolean"}
Show_VDW_surface = True #@param {type:"boolean"}
View_outline = False #@param {type:"boolean"}

#@markdown ---

#@markdown **LIGAND MODEL** \
#@markdown Enter the **< Ligand >** to be viewed.
View_ligand = "1RP_1" #@param {type:"string"}
View_experimental_ligand = "4WI_A" #@param {type:"string"}
Show_ligand = True #@param {type:"boolean"}
Show_experimental_ligand = True #@param {type:"boolean"}
Show_all_pose = False #@param {type:"boolean"}
Show_best_pose = True #@param {type:"boolean"}

#@markdown ---

#@markdown **PARTNER PROTEIN MODEL** \
#@markdown (OPTIONAL, upload to DOCKING FOLDER) \
#@markdown Enter the **< Partner Protein>** to be view.

View_partner_protein = "" #@param {type:"string"}
Show_partner_protein = False #@param {type:"boolean"}

pview_Dfile = os.path.join(DCK_FLD, f"{View_protein}.pdb")
ppview_Dfile = os.path.join(DCK_FLD, f"{View_partner_protein}.pdb")
lview_Dfile = os.path.join(DCK_FLD, f"{View_ligand[:-2]}/{View_ligand}.pdb")
eview_Dfile = os.path.join(DCK_FLD, f"{View_experimental_ligand}.pdb")
iview_Ifile = os.path.join(INT_FLD, f"{View_ligand[:-2]}/{View_ligand}_inter.csv")

VIEW_PILE(inputP=pview_Dfile,
          inputPP=ppview_Dfile,
          inputL=lview_Dfile,
          inputE=eview_Dfile,
          inputCSV=iview_Ifile,
          model=Model,
          color=Model_colour,
          focusRes=Focus_residues,
          focusResColor=Residues_colour,
          showInter=Show_interaction,
          showSubunit=Show_all_subunits,
          showResLabel=Show_residues_label,
          showVDW=Show_VDW_surface,
          showPartProt=Show_partner_protein,
          showExpLig=Show_experimental_ligand,
          showLig=Show_ligand,
          showAllPose=Show_all_pose,
          showBestPose=Show_best_pose,
          outline=View_outline)

+ Showing 7RFW_prot_A.pdb
+ Showing 4WI_A.pdb (gray)
+ Showing 1RP_1.pdb (orange)



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

---
---
# **8. Save to Google Drive**

Save the docking result into Google Drive.

In [81]:
#@title **8.1. Import Google Drive**
#@markdown This allow data to be stored in Google Drive.

# Flush and mount GDrive
with Hide():
  drive.flush_and_unmount()
  drive.mount("/content/drive", force_remount=True)

print(f"> Mounted at /content/drive")

> Mounted at /content/drive


In [82]:
#@title **8.2. Store result in Google Drive**
#@markdown Enter the **< Destination Path >** for saving. \
#@markdown This save all docking data into Google Drive.

Destination = "/content/drive/MyDrive/Docking" # @param {type:"string"}
DST_FLD = os.path.join(Destination, Job_name)
shutil.copytree(WRK_DIR, DST_FLD)

print(f"> Data saved at {DST_FLD}")

> Data saved at /content/drive/MyDrive/Docking/7RFW


# **9. Limitation**
- **Reproducibility**: please note that I used Obabel to generate 3D conformation and this stage could not be **seeded**. So just run the prepare ligand stage once only.
- **BUG UI**: some configs are still not working.
-  **BUG for PLIP** : only support under 10 conformations, but this still works, just ignore the warnings.
-  **Results extraction** : not support smina, but this you can find the unique way for smina [here](https://colab.research.google.com/github/TieuLongPhan/TieuLongPhan.github.io/blob/master/_teaching/Material/Molecular%20Docking%20labhhc/lab04-Advance_docking%20.ipynb).
- **Unfullfil pipeline**: PDBFixer and MGLTools can not work in the same environment, and google colab do not support multiple venvs.
-**Waiting time**: not display
- **Complex structure**: no interaction visualization, I'll update soon.
- **Visualization**: limited function, should use external program; note display pi-bond of experimental ligand
- **GPU support**: pro-version (email me for further discussion).
- **Virtual screening support**: pro-version (email me for further discussion).

Feel free to email me if you find any bugs. Email: [ptlong8995@gmail.com](mailto:ptlong8995@gmail.com)


